Project:

Implementation of an efficient method for construction of suffix arrays and use it for exact pattern matching.

Index

# Project: Efficient Construction of Suffix Arrays and Exact Pattern Matching

---

## 1. Explain the **prefix-doubling method** for suffix array construction.

The prefix doubling method is a deterministic algorithm for constructing the suffix array of a string by iteratively sorting suffixes according to the lexicographic order of progressively longer prefixes. 

Beginning with an array of integer ranks representing individual characters, the algorithm sorts suffix indices using rank pairs of the form `(rank_k[i], rank_k[i + 2^k] or −1)`, which encode the first `2^k` characters of each suffix; indices extending beyond the string boundary receive `−1` to preserve ordering. After each sorting stage, new compact ranks are assigned sequentially, increasing only when adjacent tuples differ, thereby reflecting the ordering of prefixes of length `2^(k+1)`. The doubling continues until every suffix acquires a distinct rank, where the sorted indices constitute the final suffix array. Correctness follows from the monotonic expansion of prefix comparisons, and the algorithm performs `O(log n)` iterations, each sorting `n` integer pairs, yielding a practical time complexity of `O(n log² n)` with comparison sorting or `O(n log n)` with radix sorting, using `O(n)` auxiliary space.

---

## 2. Testing Correctness of Suffix Array Construction

Describe how you tested your suffix array implementation.

- **Explain your tests:**  
  - Use small example strings (e.g., `"banana"`, `"abracadabra"`) where the correct suffix array is known.  
  - Compare your output against Python’s `sorted()` applied to all suffixes for validation.

- **Explain your confidence:**  
  - If your implementation matches expected results for all test cases, argue why you believe it is correct.

---

## 3. Experimental Evaluation of Running Time

Design experiments to measure the **running time** of your method.

- Compare your efficient suffix array construction with the **naive O(n² log n)** method from the mandatory project.
- Include:
  - **Best-case** inputs (e.g., strings with many repeated prefixes)
  - **Worst-case** inputs (e.g., random or diverse characters)
- Plot or tabulate the running times to show performance trends.

---

## 4. Exact Pattern Matching Using Suffix Arrays

Implement the **suffix-array–based pattern matching algorithm** using **binary search**, as presented in *Week 5*.

- Test the implementation for correctness.
- Demonstrate that the **worst-case running time** is approximately:

  \[
  O(m \times (\log n + k))
  \]

  where:
  - `m` = pattern length  
  - `n` = text length  
  - `k` = number of occurrences of the pattern  

*(Assuming the suffix array has already been constructed.)*

---

## 5. BWT-Based Exact Pattern Matching

Implement the **Burrows–Wheeler Transform (BWT)**–based pattern matching algorithm (from *Week 6*).

This involves:

- Constructing the **suffix array** (using your earlier implementation)
- Computing the **BWT** of the input string
- Building:
  - The **C-table** in **O(n)** time
  - The **O-table** in **O(σ × n)** time, where σ = alphabet size

Explain the steps and justify the time complexity of each construction phase.

---

## 6. Testing Correctness of BWT-Based Algorithm

Describe how you tested the correctness of your **BWT-based pattern matcher**.

- **Choice of test data:**
  - Include both simple and real-world texts (e.g., repetitive DNA sequences, English text).
  - Verify outputs against brute-force pattern matching or suffix-array–based results.

---

## 7. Experimental Comparison: BWT vs Suffix Array Matching

Design experiments comparing **BWT-based** and **suffix-array–based** methods.

- Examine their **running times** in practice.
- Include an experiment where:
  - Many short patterns (~100 characters)  
  - Are matched against a long text (~1,000,000 characters)
- Exclude preprocessing time (e.g., suffix array, C-table, and O-table construction)
- Discuss:
  - Efficiency differences between methods
  - How input characteristics (repetitive vs random text) affect performance

---

## Notes

- Focus on **practical runtime behavior**, not just theoretical complexity.
- Provide clear **plots**, **tables**, or **runtime statistics**.
- Include **code snippets** and **observations** to support your analysis.


## 1. Implementation of prefix-Doubling

In [1]:
def suffix_array_prefix_doubling(s: str) -> list:
    """
    Construct the suffix array of string s using the prefix-doubling method.
    
    The algorithm maintains ranks of suffix prefixes, doubling the prefix length
    each iteration by sorting tuples (rank[i], rank[i + 2^k]). Once all ranks
    are unique, the suffix order is fixed and equals the suffix array.
    
    Args:
        s: Input string (should not contain sentinel '$' already).
    
    Returns:
        List of integers representing the suffix array.
    
    Time complexity: O(n log^2 n) with Python's Timsort.
    Space complexity: O(n) for rank arrays and temporary lists.
    
    """
    
    # Append unique sentinel smaller than all characters
    s = s + '$'
    n = len(s)
    
    # Initialize rank_0 from individual characters
    rank = [ord(c) for c in s]
    sa = list(range(n))  # Current suffix indices
    k = 1  # Current prefix length exponent (2^k)
    
    while True:
        # Check if all ranks are unique; if so, we are done
        if len(set(rank)) == n:
            break
        
        # Build sort keys: (rank[i], rank[i + 2^k] or -1)
        # We sort by these pairs to order suffixes by their first 2^(k+1) chars
        keys = []
        for i in range(n):
            second = rank[i + k] if i + k < n else -1
            keys.append((rank[i], second, i))  # Include i for stable sort
        
        # Sort indices by the key pairs
        keys.sort()
        
        # Extract new order of suffix indices and assign new ranks
        new_sa = [keys[i][2] for i in range(n)]
        new_rank = [0] * n
        
        for i in range(n):
            if i > 0 and (keys[i][0], keys[i][1]) != (keys[i-1][0], keys[i-1][1]):
                new_rank[keys[i][2]] = new_rank[keys[i-1][2]] + 1
            else:
                new_rank[keys[i][2]] = new_rank[keys[i-1][2]] if i > 0 else 0
        
        rank = new_rank
        sa = new_sa
        k *= 2  # Double the prefix length for next iteration
    
    # Remove sentinel from suffix array (optional, depending on use case)
    # sa = [x for x in sa if x != n - 1]
    
    return sa

suffix_array_prefix_doubling("missisippi")




[10, 9, 6, 4, 1, 0, 8, 7, 5, 3, 2]